In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Машинный перевод. Модель seq2seq и механизм внимания

Разобраться с моделью перевода (без механизма внимания) как она устроена, запустить для перевода с русского на английский (при желании можно взять другие пары языков)
В данном практическом задании напишу и обучу модель перевода с русского на английский
на библиотеке pytorch. Добавлю в неё механизм внимания, так как это сложнее и интереснее,
а также рекуррентную сеть энкодера сделаю двунаправленной.

In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [73]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import tensorflow as tf
import unicodedata
import re
import numpy as np
import os
import io
import time
import conda

### Download and prepare the dataset

We'll use a language dataset provided by http://www.manythings.org/anki/

In [4]:
# !wget http://www.manythings.org/anki/rus-eng.zip
!wget http://www.manythings.org/anki/swe-eng.zip

--2023-05-16 08:22:04--  http://www.manythings.org/anki/swe-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 744068 (727K) [application/zip]
Saving to: ‘swe-eng.zip.3’

swe-eng.zip.3       100%[===================>] 726.63K  --.-KB/s    in 0.1s    

2023-05-16 08:22:05 (6.20 MB/s) - ‘swe-eng.zip.3’ saved [744068/744068]



In [5]:
# !mkdir rus-eng
# !unzip rus-eng.zip -d rus-eng/
!mkdir swe-eng
!unzip swe-eng -d swe-eng/

mkdir: cannot create directory ‘swe-eng’: File exists
Archive:  swe-eng.zip
replace swe-eng/_about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [32]:
!ls /kaggle/working/swe-eng -lah

total 3.1M
drwxr-xr-x 2 root root 4.0K May 16 07:47 .
drwxr-xr-x 4 root root 4.0K May 16 08:22 ..
-rw-r--r-- 1 root root 1.5K Apr  2 03:16 _about.txt
-rw-r--r-- 1 root root 3.0M Apr  2 03:16 swe.txt


In [34]:
# Download the file
path_to_file = "/kaggle/working/swe-eng/swe.txt"

In [35]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '< start > ' + w + ' <end >'
  return w

In [36]:
preprocess_sentence("I can't go.")

"< start > i can't go . <end >"

In [38]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENG, RUS]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [39]:
en, swe = create_dataset(path_to_file, None)
print(en[0])
print(swe[0])

< start > go . <end >
< start > g . <end >


In [41]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [42]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### Limit the size of the dataset to experiment faster (optional)

In [43]:
len(en), len(swe)

(23267, 23267)

In [44]:
# Try experimenting with the size of that dataset
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [45]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

18613 18613 4654 4654


In [47]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [48]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
2 ----> <
3 ----> start
1 ----> >
7 ----> jag
6 ----> r
102 ----> lite
5079 ----> uppskakad
5 ----> .
4 ----> <end
1 ----> >

Target Language; index to word mapping
3 ----> <
2 ----> start
1 ----> >
29 ----> i'm
12 ----> a
239 ----> little
862 ----> shook
63 ----> up
5 ----> .
4 ----> <end
1 ----> >


### Create a tf.data dataset

In [51]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [52]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 96]), TensorShape([64, 78]))

In [53]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=False,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [54]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_hidden = encoder(example_input_batch, sample_hidden)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (64, 1024)


In [56]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, hidden):
    # enc_output shape == (batch_size, max_length, hidden_size)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x, initial_state=hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [57]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)

In [58]:
decoder_sample_x.shape

TensorShape([64, 5865])

In [59]:
decoder_sample_h.shape

TensorShape([64, 1024])

### Define the optimizer and the loss function

In [60]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

### Checkpoints (Object-based saving)

In [75]:
checkpoint_dir = './kaggle/working/swe-eng/training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [99]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['start']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [101]:
EPOCHS = 50

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
          print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                       batch,
                                                       batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))

    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.1070
Epoch 1 Batch 100 Loss 0.1016
Epoch 1 Batch 200 Loss 0.1254
Epoch 1 Loss 0.1082
Time taken for 1 epoch 129.32504415512085 sec

Epoch 2 Batch 0 Loss 0.0675
Epoch 2 Batch 100 Loss 0.0800
Epoch 2 Batch 200 Loss 0.0777
Epoch 2 Loss 0.0775
Time taken for 1 epoch 78.51390147209167 sec

Epoch 3 Batch 0 Loss 0.0525
Epoch 3 Batch 100 Loss 0.0549
Epoch 3 Batch 200 Loss 0.0507
Epoch 3 Loss 0.0553
Time taken for 1 epoch 78.05510067939758 sec

Epoch 4 Batch 0 Loss 0.0314
Epoch 4 Batch 100 Loss 0.0401
Epoch 4 Batch 200 Loss 0.0435
Epoch 4 Loss 0.0393
Time taken for 1 epoch 78.296635389328 sec

Epoch 5 Batch 0 Loss 0.0249
Epoch 5 Batch 100 Loss 0.0266
Epoch 5 Batch 200 Loss 0.0291
Epoch 5 Loss 0.0283
Time taken for 1 epoch 77.72727608680725 sec

Epoch 6 Batch 0 Loss 0.0160
Epoch 6 Batch 100 Loss 0.0204
Epoch 6 Batch 200 Loss 0.0226
Epoch 6 Loss 0.0212
Time taken for 1 epoch 78.41791415214539 sec

Epoch 7 Batch 0 Loss 0.0200
Epoch 7 Batch 100 Loss 0.0121
Epoch 7 Batch 200 L

### Translate

* The evaluate function is similar to the training loop, except we don't use teacher forcing here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the end token.
* And store the attention weights for every time step.

Note: The encoder output is calculated only once for one input.

In [124]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['start']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden)

        # storing the attention weights to plot later on
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == 'end':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [125]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

### Restore the latest checkpoint and test

In [126]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [127]:
translate('Det är trevligt här.')

Input: < start > det r trevligt h r . <end >
Predicted translation: start > this is the matter . <end > over is here . <end > it is on fire . <end > over it is . <end > it is on monday . <end > it was on fire . <end > it is out of them . <end > over one hundred . <end > it is a try . <end > it was on a front of time . <end > it is on fire . 


In [128]:
translate('Jag kommer inte att kunna gå.')

Input: < start > jag kommer inte att kunna g . <end >
Predicted translation: start > i'm not going to go . <end > i'm here . <end > i'm here . <end > do you want ? <end > it is fun . <end > over you . <end > might do the matter . <end > over one break . <end > over you ready ? <end > it was around for you . <end > it is a short designed for me to look . <end > it is 


In [129]:
translate(u'Är du fortfarande hemma?')

Input: < start > r du fortfarande hemma ? <end >
Predicted translation: start > are you still at home ? <end > it could be right . <end > it was under night . <end > it was a day . <end > it's on better . <end > does a practice of them are around . <end > it was on for a list . <end > it is on a man . <end > it is a mistake on their first floor . <end > it does . 


In [130]:
translate(u'Är du fortfarande hemma?')

Input: < start > r du fortfarande hemma ? <end >
Predicted translation: start > are you still at home ? <end > it could be right . <end > it was under night . <end > it was a day . <end > it's on better . <end > does a practice of them are around . <end > it was on for a list . <end > it is on a man . <end > it is a mistake on their first floor . <end > it does . 


In [131]:
translate(u'Prova detta.')

Input: < start > prova detta . <end >
Predicted translation: start > don't do it . <end > this is black . <end > it was seems to be . <end > it set himself . <end > it was getting first . <end > it was raining heavily . <end > it is about a look designed for him . <end > it was about it . <end > it is about that purpose such over . <end > it is . <end > it was seems 


In [132]:
translate(u'Jag älskar det när det snöar.')

Input: < start > jag lskar det n r det sn ar . <end >
Predicted translation: start > i love that dress . <end > it was that way that one giant leap for mankind . <end > it is a difficult own mistake . <end > it was raining heavily . <end > it is about a dream . <end > it is a look at white . <end > it is as a look designed for over . <end > it is a difficult way . <end > it is a difficult 


In [133]:
translate(u'Jag gör aldrig det.')

Input: < start > jag g r aldrig det . <end >
Predicted translation: start > i never do it . <end > it does . <end > it is a difficult question . <end > it is a way . <end > it is . <end > tom is over a car . <end > it might . <end > it is a look . <end > it was saying . <end > it is . <end > it's a better designed for that purpose such a website designed for them 
